# Description of Architectue

### <a id="top"></a>Contents
* [Overview](#overview)
* [Architectural Element](#element)
* [Architectural Block](#block)
* [An Illustative Example](#example)

## <a id="overview"></a>Overview

The architecture description employs a hierarchical design and a flattened implementation.

An architectural component is described as an [element](#element) or a [block](#block).
* An element is a leaf in the architectural hierarchy.
* A block consists of elements and/or blocks to form a hierarchical architecture.

### Setup

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

In [ ]:
import pillars.core.{BlockTrait, OpEnum}
import pillars.core.OpEnum.OpEnum
import pillars.hardware.PillarsConfig
import pillars.archlib.{ElementAlu, ElementConst, ElementMux}
import pillars.archlib.TileLSUBlock
import pillars.core._
import pillars.hardware.{TopModule}
import pillars.archlib._

## <a id="element"></a>Custom Architecture

User can build a custom architecture with "Element" and "Block".

### **Element**

Element corresponds to a Chisel hardware implementation.

Each element has predefined lowering rules to generate an IR for hardware generation (e.g., Chisel modules) and mapping algorithms (e.g., MRRG or TEC). The domain knowledge is codified in these lowering rules.

There are five predefined elements: 
* multiplexer
* const unit
* arithmetic logical unit
* load/store unit (LSU)
* register files (RF).

### **Block**

Blocks represent the design hierarchy. A block can be composed of several sub-blocks and elements.

Main methods of a block:
* initName(name): set the name of this block
* addInPorts(ports): add some input ports of this block
* addOutPorts(ports): add some output ports of this block
* addElement(element): add an element in this block
* addBlock(element): add a sub-block in this block
* term(portName): get a port of this block
* addConnect(port -> port): connet ports within this block
* setConfigRegion(): set this block as a reconfiguration region.

<div style="color: red;">NOTICE:</div>All blocks and elements are identified by names, and if they share a collective parent block, their name must be different.





## Example

### Example1: Define a block (`SimpleBlock`) with a single ALU element.

In [ ]:
class SimpleBlock(name: String, isRegion: Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0","input_1"))
    addOutPorts(Array("out_0"))
    
    if(isRegion){
        setConfigRegion()
    }
    
    // Initialize ALU supporting ADD/SUB
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB)
    val supBypass = false 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, List(32))
    alu0.addInPorts(Array("input_A", "input_B"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)
    
    // Initialize internal connections
    addConnect(term("input_0") -> alu0 / "input_A")
    addConnect(term("input_1") -> alu0 / "input_B")
    addConnect(alu0 / "out_0" -> term("out_0"))
}

### Example2: Define a block (`BlockImmediate`) with three elements

The block performs computation between the selected input and an immediate operand.

In [ ]:
/** A subblock that performs computation between the selected input and a immediate operand.
 *
 * @constructor create an abstract block model that performs computation between the input and a immediate operand
 * @param name the name of the model
 */
class BlockImmediate(name: String) extends BlockTrait {
  val aluParams = List(32)
  val aluOpList = List(OpEnum.ADD)
  val muxParams = List(2, 32)
  val constParams = List(PillarsConfig.CONST_WIDTH)
  initName(name)

  addInPorts(Array("in0", "in1"))
  addOutPorts(Array("out0"))
  setConfigRegion()

  /** A multiplexer that can choose a data source for the port "inputA" of the ALU.
   */
  val mux0 = new ElementMux("mux0", muxParams)
  mux0.addInPorts(Array("input0", "input1"))
  mux0.addOutPorts(Array("out0"))
  addElement(mux0)

  /** An ALU that can perform some operations.
   */
  val alu0 = new ElementAlu("alu0", aluOpList, supBypass = true, aluParams)
  alu0.addInPorts(Array("inputA", "inputB"))
  alu0.addOutPorts(Array("out0"))
  addElement(alu0)

  /** A const unit connected to the port "inputB" of ALU.
   */
  val const0 = new ElementConst("const0", constParams)
  const0.addOutPorts(Array("out0"))
  addElement(const0)

  /** Interconnection inside this block.
   */
  addConnect(term("in0") -> mux0 / "input0")
  addConnect(term("in1") -> mux0 / "input1")
  addConnect(mux0 / "out0" -> alu0 / "inputA")
  addConnect(const0 / "out0" -> alu0 / "inputB")
  addConnect(alu0 / "out0" -> term("out0"))
}

### Example3: Define a block (`NestBlock`) with a simple block and some other elements


In [ ]:
class NestBlock(name: String, isRegion:Boolean = false) extends BlockTrait {
    initName(name)
    addInPorts(Array("input_0", "input_1"))
    addOutPorts(Array("out_0"))

     if(isRegion){
        setConfigRegion()
    }
    
    // A multiplexer that can choose a data source for the port "inputA" of the ALU.
    val muxParams = List(2, 32) // 2-input 32-bit width
    val mux0 = new ElementMux(name+"_mux", muxParams)
    mux0.addInPorts(Array("input_0", "input_1"))
    mux0.addOutPorts(Array("out_0"))
    addElement(mux0)
    
    // An ALU that can perform some operations.
    val aluOpList = List(OpEnum.ADD, OpEnum.SUB, OpEnum.AND, OpEnum.OR, OpEnum.XOR, OpEnum.MUL)
    val supBypass = true 
    val aluParams = List(32) // 32 bit width
    val alu0 = new ElementAlu(name+"_ALU", aluOpList, supBypass, aluParams)
    
    alu0.addInPorts(Array("inputA", "inputB"))
    alu0.addOutPorts(Array("out_0"))
    addElement(alu0)
    
    // A const unit connected to the port "inputB" of ALU.
    val constParams = List(32) // 32 bit width
    val const0 = new ElementConst(name+"_const0", constParams)
    const0.addOutPorts(Array("out_0"))
    addElement(const0)
    
    // A simple sub-block with 2 input ports and 1 output port.
    val subBLock = new SimpleBlock(name+"_subBlock")
    addBlock(subBLock)

    // Interconnection inside this block.
    addConnect(term("input_0") -> mux0 / "input_0")
    addConnect(term("input_1") -> mux0 / "input_1")
    addConnect(mux0 / "out_0" -> alu0 / "inputA")
    addConnect(const0 / "out_0" -> alu0 / "inputB")
    addConnect(term("input_1") -> subBLock / "input_0")
    addConnect(alu0 / "out_0" -> subBLock / "input_1")
    addConnect(subBLock / "out_0" -> term("out_0"))
}

**See** [Generation of Hardware](5-gen-hardware.ipynb) to continue generating Verilog code.